In [ ]:
# Код для обучения гибридной модели LSTM+Автоэнкодер
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Concatenate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

# Загрузка данных
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

# Подготовка данных
def prepare_data(data, sequence_length=10):
    # Разделение на признаки и метки
    X = data.drop('label', axis=1)
    y = data['label']
    
    # Масштабирование данных
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Создание последовательностей для LSTM
    X_sequences = []
    y_sequences = []
    
    for i in range(len(X_scaled) - sequence_length):
        X_sequences.append(X_scaled[i:i+sequence_length])
        y_sequences.append(y.iloc[i+sequence_length])
    
    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)
    
    # Подготовка данных для автоэнкодера (текущий временной шаг)
    X_current = np.array([seq[-1] for seq in X_sequences])
    
    # Разделение на обучающую и тестовую выборки
    X_train_seq, X_test_seq, X_train_cur, X_test_cur, y_train, y_test = train_test_split(
        X_sequences, X_current, y_sequences, test_size=0.2, random_state=42
    )
    
    return X_train_seq, X_test_seq, X_train_cur, X_test_cur, y_train, y_test, scaler

# Создание автоэнкодера
def create_autoencoder(input_dim):
    # Кодировщик
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(16, activation='relu')(encoded)
    
    # Декодировщик
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='linear')(decoded)
    
    # Полная модель автоэнкодера
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    # Модель кодировщика для извлечения признаков
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    return autoencoder, encoder

# Создание модели LSTM
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu')
    ])
    return model

# Создание гибридной модели
def create_hybrid_model(lstm_input_shape, autoencoder_input_dim):
    # Создаем компоненты
    autoencoder, encoder = create_autoencoder(autoencoder_input_dim)
    lstm_model = create_lstm_model(lstm_input_shape)
    
    # Входы для обеих моделей
    lstm_input = Input(shape=lstm_input_shape)
    autoencoder_input = Input(shape=(autoencoder_input_dim,))
    
    # Получаем выходы от компонентов
    lstm_features = lstm_model(lstm_input)
    encoder_features = encoder(autoencoder_input)
    
    # Объединяем признаки
    combined = Concatenate()([lstm_features, encoder_features])
    
    # Выходной слой
    output = Dense(1, activation='sigmoid')(combined)
    
    # Создаем гибридную модель
    hybrid_model = Model(inputs=[lstm_input, autoencoder_input], outputs=output)
    
    # Компилируем модель
    hybrid_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return hybrid_model, autoencoder

# Обучение моделей
def train_models(hybrid_model, autoencoder, X_train_seq, X_train_cur, X_test_seq, X_test_cur, y_train, y_test):
    # Сначала обучаем автоэнкодер
    autoencoder_history = autoencoder.fit(
        X_train_cur, X_train_cur,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_cur, X_test_cur),
        verbose=1
    )
    
    # Затем обучаем гибридную модель
    hybrid_history = hybrid_model.fit(
        [X_train_seq, X_train_cur], y_train,
        epochs=30,
        batch_size=32,
        validation_data=([X_test_seq, X_test_cur], y_test),
        verbose=1
    )
    
    return autoencoder_history, hybrid_history

# Оценка результатов
def evaluate_hybrid(hybrid_model, X_test_seq, X_test_cur, y_test):
    # Предсказание
    y_pred_prob = hybrid_model.predict([X_test_seq, X_test_cur])
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Оценка точности
    accuracy = accuracy_score(y_test, y_pred)
    
    return y_pred, y_pred_prob, accuracy

# Визуализация результатов
def plot_results(autoencoder_history, hybrid_history, y_test, y_pred):
    # График потерь автоэнкодера
    plt.figure(figsize=(12, 4))
    plt.plot(autoencoder_history.history['loss'], label='Обучение')
    plt.plot(autoencoder_history.history['val_loss'], label='Валидация')
    plt.title('Потери автоэнкодера')
    plt.xlabel('Эпоха')
    plt.ylabel('Потери')
    plt.legend()
    plt.savefig('hybrid_ae_loss.png')
    
    # График потерь гибридной модели
    plt.figure(figsize=(12, 4))
    plt.plot(hybrid_history.history['loss'], label='Обучение')
    plt.plot(hybrid_history.history['val_loss'], label='Валидация')
    plt.title('Потери гибридной модели')
    plt.xlabel('Эпоха')
    plt.ylabel('Потери')
    plt.legend()
    plt.savefig('hybrid_loss.png')
    
    # Матрица ошибок
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Матрица ошибок: Гибридная модель')
    plt.ylabel('Истинный класс')
    plt.xlabel('Предсказанный класс')
    plt.savefig('hybrid_confusion_matrix.png')

# Основная функция
def main():
    data = load_data('arp_dataset.csv')
    X_train_seq, X_test_seq, X_train_cur, X_test_cur, y_train, y_test, scaler = prepare_data(data)
    
    # Создание моделей
    hybrid_model, autoencoder = create_hybrid_model(
        (X_train_seq.shape[1], X_train_seq.shape[2]), 
        X_train_cur.shape[1]
    )
    
    # Обучение моделей
    autoencoder_history, hybrid_history = train_models(
        hybrid_model, autoencoder, 
        X_train_seq, X_train_cur, X_test_seq, X_test_cur, 
        y_train, y_test
    )
    
    # Оценка модели
    y_pred, y_pred_prob, accuracy = evaluate_hybrid(
        hybrid_model, X_test_seq, X_test_cur, y_test
    )
    print(f"Точность гибридной модели: {accuracy:.4f}")
    
    # Визуализация результатов
    plot_results(autoencoder_history, hybrid_history, y_test, y_pred)
    
    # Сохранение моделей
    hybrid_model.save('models/hybrid_model.h5')
    autoencoder.save('models/hybrid_autoencoder.h5')

if __name__ == "__main__":
    main()